<a href="https://colab.research.google.com/github/DarshanPatel0919/Computational-Finance/blob/main/LAB5_201701436.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pylab as plt
import math
import scipy.stats as si

## Q 1 : Barrier Option

In [ ]:
def BSM(x,t,T,k,r,sig):
    tau=T-t
    d1=(np.log(x/k) + (r+0.5*(sig*sig))*tau)/(sig*np.sqrt(tau))
    d2=d1-(sig*np.sqrt(tau))
    bsm = x*si.norm.cdf(d1,0.0,1.0) -k*np.exp(-r*tau)*si.norm.cdf(d2,0.0,1.0)
    print('BSM Price :',bsm,'\n')
def MonteCarlo_Antithetic_BarrierOption(K,n,N,sig,r,s0,V):
    Sin=np.zeros(K)
    Sout=np.zeros(K)
    Sinn=np.zeros(K)
    Soutt=np.zeros(K) 
    std = sig/np.sqrt(N) 
    mu=(r-0.5*(std*std*N))/N

    for j in range(K):
        np.random.seed(j+1)
        xi = np.random.normal(mu,std,n)
        S,DIi,DIo=[s0],0,1
        for i in range(n):
            S.append(S[-1]*np.exp(xi[i]))
            if (S[-1]<=V):DIi,DIo=1,0
        Sin[j]=np.exp(-r*n/N)*DIi*max(S[-1]-k,0)
        Sout[j]=np.exp(-r*n/N)*DIo*max(S[-1]-k,0)

        S,DIi,DIo=[s0],0,1
        for i in range(n): 
            S.append(S[-1]*np.exp(2*mu-xi[i]))
            if (S[-1]<=V):DIi,DIo=1,0
        Sinn[j]=np.exp(-r*n/N)*DIi*max(S[-1]-k,0)
        Soutt[j]=np.exp(-r*n/N)*DIo*max(S[-1]-k,0)
        

    din=np.mean(Sin)
    dout=np.mean(Sout)
    print('Simulations = ',K,'\nMonte Carlo\n',' Down and in barrier option value : ',din,'\n  Down and out barrier option value: ',dout)
    print('  Sum of both barrier option values: ',din+dout,'\n')
    din=np.mean((Sinn+Sin)/2)
    dout=np.mean((Soutt+Sout)/2)
    print('Antithetic variable \n  Down and in barrier option value : ',din,'\n  Down and out barrier option value: ',dout)
    print('  Sum of both barrier option values: ',din+dout,'\n\n')

In [ ]:
k,r,sig,n,V,N,s0=50,5e-2,0.3,252,45,252,50
BSM(50,0,1,k,r,sig)
for i in range(2,6):MonteCarlo_Antithetic_BarrierOption(10**i,n,N,sig,r,s0,V)

BSM Price : 7.115627392992909 

Simulations =  100 
Monte Carlo
  Down and in barrier option value :  1.7064282987045478 
  Down and out barrier option value:  4.69656370357911
  Sum of both barrier option values:  6.402992002283658 

Antithetic variable 
  Down and in barrier option value :  1.605512829561335 
  Down and out barrier option value:  5.11126650541125
  Sum of both barrier option values:  6.7167793349725855 


Simulations =  1000 
Monte Carlo
  Down and in barrier option value :  2.194800905518483 
  Down and out barrier option value:  5.195233463992542
  Sum of both barrier option values:  7.390034369511025 

Antithetic variable 
  Down and in barrier option value :  2.021693516293855 
  Down and out barrier option value:  4.989346475694626
  Sum of both barrier option values:  7.011039991988481 


Simulations =  10000 
Monte Carlo
  Down and in barrier option value :  2.2089038383863384 
  Down and out barrier option value:  5.106335650417762
  Sum of both barrier optio

## Q 2 Asian Option

In [ ]:
def MonteCarlo_Controlvariate_Asian(K,n,N,sig,r,s0):
    Sin=np.zeros(K)
    V=np.zeros(K)
    UV=np.zeros(K)
    std = sig/np.sqrt(N)
    mu=(r-0.5*(std*std*N))/N
    
    for j in range(K):
        np.random.seed(j+1)
        xi = np.random.normal(mu,std,n)
        S=[s0]
        for i in range(n):S.append(S[-1]*np.exp(xi[i]))
        Sin[j]=np.exp(-r*n/N)*max(np.mean(S)-k,0)
        V[j]=np.sum(S)
        UV[j]=S[0]*(1-np.exp(r*(n+1)/N))/(1-np.exp(r/N))
    
    COV=np.cov(Sin,V)
    C=-COV[0][1]/(np.var(V))
    Z=Sin+C*(V-UV)
    asian=np.mean(Sin)
    print('Simulations',K,'\nMonte Carlo Asian option Value: ',asian)
    asian=np.mean(Z)
    print('Controlvariate Asian option Value  : ',asian,'\n\n')

In [ ]:
k,r,sig,n,N,s0=100,6e-2,0.25,252,252,100
for i in range(2,6):MonteCarlo_Controlvariate_Asian(10**i,n,N,sig,r,s0)

Simulations 100 
Monte Carlo Asian option Value:  6.650613853850224
Controlvariate Asian option Value  :  7.076515997564274 


Simulations 1000 
Monte Carlo Asian option Value:  7.526999973780151
Controlvariate Asian option Value  :  7.119151230303046 


Simulations 10000 
Monte Carlo Asian option Value:  7.25934377806506
Controlvariate Asian option Value  :  7.110973120153497 


Simulations 100000 
Monte Carlo Asian option Value:  7.04945957816357
Controlvariate Asian option Value  :  7.0958955942363025 




## Q 3 : Loop-back Option

In [ ]:
def MonteCarlo_Loopback(K,n,N,sig,r,s0):
    Sin=np.zeros(K)
    std = sig/np.sqrt(N)
    mu=(r-0.5*(std*std*N))/N
    
    for j in range(K):
        np.random.seed(j+1)
        xi = np.random.normal(mu,std,n)
        S=[s0]
        for i in range(n):S.append(S[-1]*np.exp(xi[i]))
        Sin[j]=np.exp(-r*n/N)*max(S[-1]-np.min(S),0)
    lpbk=np.mean(Sin)
    print('Simulations :',K)
    print('Loop-back Option Value:',lpbk)

In [ ]:
k,r,sig,n,N,s0=100,6e-2,0.25,252,252,100
for i in range(2,6):MonteCarlo_Loopback(10**i,n,N,sig,r,s0)

Simulations : 100
Loop-back Option Value: 19.148586771415104
Simulations : 1000
Loop-back Option Value: 20.736241020212002
Simulations : 10000
Loop-back Option Value: 20.5565471854992
Simulations : 100000
Loop-back Option Value: 20.26079791710138
